# Laboratorio 3
Diana Díaz 21066
Mariel Guamuche 21150

## Configuración del entorno
1.1 Instalación de Dependencias 

In [ ]:
!pip install mlflow feast scikit-learn pandas numpy matplotlib seaborn

1.2 Verificación de instalación

In [ ]:
import mlflow
import feast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

print("mlflow.__version__", mlflow.__version__)
print("feast.__version__", feast.__version__)

## 2 Carga y exploración de datos

2.1 Carga del dataset

In [ ]:
from sklearn.datasets import fetch_california_housing
# Cargar el dataset de California Housing
data = fetch_california_housing(as_frame=True)
df = data.frame
# Mostrar las primeras filas del DataFrame
df.head()

In [ ]:
# Convertir las columnas con nombre de columnas
df.rename(columns={
    'MedInc': 'ingresos_medios',
    'HouseAge': 'promedio_edad_casas',
    'AveRooms': 'promedio_num_habitaciones',
    'AveBedrms': 'promedio_num_dormitorios',
    'Population': 'poblacion_distrito',
    'AveOccup': 'promedio_personas_casa',
    'Latitude': 'latitud',
    'Longitude': 'longitude',
    'MedHouseVal': 'target_valor_medio_casa'
}, inplace=True)   
# Mostrar las primeras filas del DataFrame con nombres modificados
df.head()

In [ ]:
# información básica del DataFrame
df.shape

In [ ]:
# información básica del DataFrame
df.info()

Los tipos de datos son congruentes a las variables

2.2 Análisis exploratorio

In [ ]:
# descripción estadística del DataFrame
df.describe()

In [ ]:
# Cantidad de valores nulos por columna
df.isnull().sum()

El dataset pareciera no presentar problemas. Hay que verificar la distribución en la cantidad de habitaciones y dormitorios, ya que pareciera haber outlayers

In [ ]:
# Boxplots para detectar outliers
plt.figure(figsize=(15, 10))
for i, column in enumerate(df.columns, 1):
    plt.subplot(3, 3, i)
    sns.boxplot(y=df[column])
    plt.title(f'Boxplot de {column}')
plt.tight_layout()
plt.show()

- Los ingresos_medios presentan una distribución sesgada a la derecha; hay outlayers a partir de 10K USD.
- promedio_edad_casas presenta muchos outliers; presentando localizaciones con más de 100 habitaciones en promedio.
- promedio_num_dormitorios presenta sesgo, con outliers al igual que la variable anterior.
- poblacion_distriro alta concentración en valores bajos y algunso distritos con poblaciones con alta concentración de personas.
- promedio_personas_casa presenta sesgo, con valores poco razonables de la cantidad de personas que pueden habitar en una vivienda.
- target_valor_medio_casas: distribución moderadamente sesgada a la derecha, con pocos outliers.

Es evidente que 5 de las 9 variables tienen outlayers, dependiendo de los modelos podría convenir eliminar los outliers en especial con el de regresión lineal. Se realizará este filtrado en las variables de habitaciones y de población 

In [ ]:
# Matriz de correlación
plt.figure(figsize=(10, 8))
correlation_matrix = df.corr() 
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlación')
plt.show()

Las relaciones entre las variables, son fuerte entre las de carácter económico (ingresos_medios y target_valor_medio_casa), las de localización geográfica (latitud y longitud) y las de habitaciones (promedio_num_habitaciones y promedio_num_dormitorios)

## 3. Preparación de datos
3.0 Eliminación de outliers

In [ ]:
filt = (
    (df["promedio_num_habitaciones"] < 50) &   # evitar promedios irreales
    (df["promedio_personas_casa"]   < 15) &    # evitar divisiones pequeñas
    (df["promedio_num_dormitorios"] < 10)      # por coherencia con habitaciones
)
df_clean = df.loc[filt].reset_index(drop=True)
print(df.shape, "->", df_clean.shape)
df_clean.describe()

3.1 Feature engineering
Las variables que implementarán son:
- ratio_habitaciones_hogar indica el tamaño medio del hogar por persona (espacio disponible), se pensaría que las zonas con más espacio por persona tienen precios más altos
- ratio_habitaciones_poblacion es la densidad de habitaciones por cantidad total de personas.
- ratio_dormitorios_habitacion es la proporción de dormitorios respecto a habitaciones totales
- ingreso_persona es el nivel de ingreso ajustado por cantidad promedio de personas

In [ ]:
# Ratios espaciales y socioeconómicos
df_clean["ratio_habitaciones_hogar"] = (
    df_clean["promedio_num_habitaciones"] / df_clean["promedio_personas_casa"]
)

df_clean["ratio_habitaciones_poblacion"] = (
    df_clean["promedio_num_habitaciones"] / df_clean["poblacion_distrito"]
)

df_clean["ratio_dormitorios_habitacion"] = (
    df_clean["promedio_num_dormitorios"] / df_clean["promedio_num_habitaciones"]
)

df_clean["ingreso_persona"] = (
    df_clean["ingresos_medios"] / df_clean["promedio_personas_casa"]
)

# Reemplazar valores infinitos o nulos por mediana
df_clean.replace([np.inf, -np.inf], np.nan, inplace=True)
df_clean.fillna(df_clean.median(numeric_only=True), inplace=True)

3.2 División de datos

Este punto se ha realizado después y no antes de definir las features por simplicidad

In [ ]:
from sklearn.model_selection import train_test_split
# Separar características y variable objetivo
X = df_clean.drop('target_valor_medio_casa', axis=1)
y = df_clean['target_valor_medio_casa']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)
print("Tamaño del conjunto de entrenamiento (y):", y_train.shape)
print("Tamaño del conjunto de prueba (y):", y_test.shape)

## 4. Experimentación con MLFlow
4.1 Configuración de MLFlow

Se ha decidido conectar con databricks

In [ ]:
import mlflow.sklearn
import mlflow

# configurar tracking URI de MLFlow
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
print("Tracking URI:", mlflow.get_tracking_uri())

In [ ]:
# Crear el experimento 
mlflow.set_experiment("california-housing-prediction")

4.2 Entrenamiento de Modelos

Se utilizarán los modelos de regresión lineal, random forest regressor y gradient boosting regressor

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

# Función de entrenamiento y logging
def train_and_log(model, model_name, params=None):
    with mlflow.start_run(run_name=model_name):
        if params:
            mlflow.log_params(params)
        model.fit(X_train, y_train)
        preds = model.predict(X_test)

        metrics = {
            "RMSE": root_mean_squared_error(y_test, preds),
            "MAE": mean_absolute_error(y_test, preds),
            "R2": r2_score(y_test, preds)
        }

        mlflow.log_metrics(metrics)
        mlflow.sklearn.log_model(model, model_name)
        print(f"{model_name} ->", metrics)
        return model, metrics

# Entrenamiento de tres modelos
lin_params = {}
rf_params = {"n_estimators": 300, "max_depth": 12, "random_state": 42}
gb_params = {"n_estimators": 300, "learning_rate": 0.05, "max_depth": 3, "random_state": 42}

model_lin, metrics_lin = train_and_log(LinearRegression(), "LinearRegression", lin_params)
model_rf, metrics_rf = train_and_log(RandomForestRegressor(**rf_params), "RandomForestRegressor", rf_params)
model_gb, metrics_gb = train_and_log(GradientBoostingRegressor(**gb_params), "GradientBoostingRegressor", gb_params)